In [137]:
# Calibration with Fit
import numpy as npy
import scipy.optimize as opt
import matplotlib.pyplot as plt

class currentProfile:
    def __init__(self, label):
        self.label=str(label)
        self.current=[]

def dataReader(filename, label):
    inputfile=open(filename,'r')
    outputData=currentProfile(label)
    currentData=[]
    while(1):
        try:
            temp=next(inputfile).split()
            if (temp[4] == "test"):
                continue
            if((isinstance(label, str) and temp[4]==str(label)) or (isinstance(label, float) and not temp[4]=="dark" and float(temp[4])==label)):
                outputData.current.append(1e6*float(temp[2]))
        except StopIteration:
            outputData.current=npy.array(outputData.current)
            break
    return outputData

def fitFunc(time, omega, gamma, tanPsi, cosDelta):
    # 47 deg
    Ps=-0.060052071038013966+0.8203047484373349
    # 46 deg
    # Ps=-0.060052071038013966+0.8028514559173915
    As=-0.030955717123416913
    eta=1.1368132418974821
    # eta=1.4
    f0=npy.cos(Ps)**2 * tanPsi**2 +  npy.sin(Ps)**2
    f1=npy.cos(Ps)**2 * tanPsi**2 -  npy.sin(Ps)**2
    f2=npy.sin(2*Ps)*tanPsi*cosDelta
    return gamma*(f0*eta+f1*npy.cos(2*(omega*time+As))+f2*npy.sin(2*(omega*time+As)))

def alphaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal = (tanPsi**2 - npy.tan(P-Ps)**2) / (tanPsi**2 + npy.tan(P-Ps)**2)
    betaReal = (2 * tanPsi * cosDelta * npy.tan(P-Ps)) / \
        (tanPsi**2 + npy.tan(P-Ps)**2)
    return (alphaReal*npy.cos(2*As)-betaReal*npy.sin(2*As))/eta


def betaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal = (tanPsi**2 - npy.tan(P-Ps)**2) / (tanPsi**2 + npy.tan(P-Ps)**2)
    betaReal = (2 * tanPsi * cosDelta * npy.tan(P-Ps)) / \
        (tanPsi**2 + npy.tan(P-Ps)**2)
    return (alphaReal*npy.sin(2*As)+betaReal*npy.cos(2*As))/eta


def residualCurve(P, Ps, As, tanPsi, cosDelta, eta):
    return 1-alphaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2-betaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2


def fitCurve(P, Ps, As, tanPsi, cosDelta, eta):
    length=int(len(P))
    P=P[:length]
    return residualCurve(P, Ps, As, tanPsi, cosDelta, eta)

def fitCurve2(P, Ps, As, tanPsi, cosDelta, eta):
    length=int(len(P)/2)
    P=P[:length]
    return npy.append(alphaCurve(P, Ps, As, tanPsi, cosDelta, eta), betaCurve(P, Ps, As, tanPsi, cosDelta, eta))

def fitCurve3(P, Ps, As, tanPsi, cosDelta, eta):
    length=int(len(P)/3)
    P=P[:length]
    return npy.append(alphaCurve(P, Ps, As, tanPsi, cosDelta, eta), npy.append(betaCurve(P, Ps, As, tanPsi, cosDelta, eta), residualCurve(P, Ps, As, tanPsi, cosDelta, eta)))

def refractionRate(tanPsi, cosDelta):
    rho = tanPsi*(cosDelta+1j*npy.sqrt(1-cosDelta**2))
    iAngle=90-27.35
    # iAngle=90-27.45 BEFORE
    # LAB 0413 27.40 27.35 27.35
    # LAB 27.45
    # QUARTZ 27.16
    return npy.sqrt(npy.sin(iAngle/180.0*npy.pi)**2*(1+(npy.tan(iAngle/180.0*npy.pi))**2*((1-rho)/(1+rho))**2))


In [138]:
# index=[str(420+20*i)+"t"+str(j) for i in range(int((600-420)/20+1)) for j in range(4)]
index=[str(400+10*i)+"t"+str(j) for i in range(int((420-400)/10+1)) for j in range(2)]
print(index)
# angle=
length=len(index)
measurements=[]
monitors=[]

for i in range(length):
    measurements.append(dataReader("dataSi/0416Si/measurement.txt",index[i]))
    monitors.append(dataReader("dataSi/0416Si/monitor.txt", index[i]))
    # measurements.append(dataReader("dataLAB/0413lab/measurement400nm.txt",index[i]))
    # monitors.append(dataReader("dataLAB/0413lab/monitor400nm.txt", index[i]))
    # measurements.append(dataReader("dataLAB/measurement.txt",index[i]))
    # monitors.append(dataReader("dataLAB/monitor.txt", index[i]))

measurementDark=dataReader("dataQuartz/0410Quartz/measurement.txt", "dark")
monitorDark=dataReader("dataQuartz/0410Quartz/monitor.txt", "dark")

measurementDarkAverage=npy.mean(measurementDark.current)*1.1

['400t0', '400t1', '410t0', '410t1', '420t0', '420t1']


In [139]:
ratios=[]
fitParameters=[]
fitParametersCOV=[]

# Data-taking 8s for single A
anglesA=npy.array([float(10*i)/180*npy.pi for i in range(1,37)])
# 6*36
minLength=450
period=12
modification=0.27
translate=2
omit=1
time=npy.array([0.5*j for j in range(minLength)])
for i in range(length):
    print(i)
    ratios.append((measurements[i].current[:minLength]-measurementDarkAverage)/monitors[i].current[:minLength])
    value=[]
    fig,axs=plt.subplots(2,1)
    # axs[0].plot(time, ratios[i],label="Data")
    for j in range(len(anglesA)):
        value.append(npy.mean(ratios[i][translate+int((period+modification)*j):translate+int((period+modification)*(j+1))][omit:-omit]))
        # value=[npy.mean(ratios[i][translate+int((period+modification)*j):translate+int((period+modification)*(j+1))][omit:-omit]) for k in range(len(time[translate+int((period+modification)*j):translate+int((period+modification)*(j+1))][omit:-omit]))]
        # axs[0].plot(time[translate+int((period+modification)*j):translate+int((period+modification)*(j+1))][omit:-omit],value)
  
    # fitFunc(time, omega, gamma, tanPsi, cosDelta):
    popt,pcov=opt.curve_fit(fitFunc, anglesA, value, p0=[1, 23,  0.5, 0.9], maxfev=1000000,bounds=([0.9,10,-1,0],[1.1,40,1,1]))
    fitParameters.append(popt)
    fitParametersCOV.append(pcov)
    fitRatio=fitFunc(anglesA,*popt)
    axs[0].plot(anglesA, value,label="Data")
    axs[0].plot(anglesA,fitRatio,label="Fitted")
    axs[1].plot(anglesA,(value-fitRatio)/value,".",label="Residual")
    axs[1].set_xlabel("Time (s)")
    axs[0].set_ylabel("Intensity")
    axs[1].set_ylabel("Residual")
    axs[0].legend()
    axs[1].legend()
    fig.tight_layout()
    fig.savefig("figure/RatioVsAng"+measurements[i].label+".jpg",dpi=300)
    plt.close()
# print(npy.mean(npy.array([fitParameters[i][2] for i in range(length)])))

0
1
2
3
4
5


In [140]:
nValues=[]
label=[str(400+10*i) for i in range(int((420-400)/10+1))]
for i in range(len(label)):
    ni=[refractionRate(fitParameters[int(2*i+j)][2],fitParameters[int(2*i+j)][3]) for j in range(2)]
    # nError=refractionRate(iPara[2],iPara[3])
    print(str(label[i]) + "nm, n:"+str(round(npy.real(npy.mean(ni)),4))+ "  k:"+str(npy.imag(npy.mean(ni))))
    nValues.append(npy.mean(ni))

400nm, n:5.4303  k:1.1021878442633502e-07
410nm, n:5.1787  k:9.865954097375102e-08
420nm, n:4.979  k:8.988114663608444e-08



n=1.505274469962087 27.45

n=1.5063188587617735 27.35